In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell

In [10]:
module_path

'C:\\Users\\eadel\\Documents\\DDMSTools'

In [19]:
from flowability_data_upload.local.generate_datasets.loaddata import load_data
import flowability_data_upload.local.preprocess as preprocess

In [18]:
from main import visualize_correlation

ModuleNotFoundError: No module named 'main'

In [ ]:
from flowability

In [20]:
from ipynb.fs.full.DataAnalysis import factorial

ModuleNotFoundError: No module named 'ipynb'

In [22]:
def makemodel(df, y = "Flow"):
    #print(df)
    
    df_features = df.copy()
    df_labels = df_features.pop(y)
    #print(df_labels)
    #df_features = df.drop(columns = y)
    df_features = np.array(df_features)
    #print(len(df_features))
    df_features
    
    df_model = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
    ])

    df_model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())
    df_model.fit(df_features, df_labels, epochs=1)
    
    normalize = preprocessing.Normalization()
    
    normalize.adapt(df_features)
    
    norm_df_model = tf.keras.Sequential([
    normalize,
    layers.Dense(64),
    layers.Dense(1)
    ])

    norm_df_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam())

    norm_df_model.fit(df_features, df_labels, epochs=10)
    
    return norm_df_model

In [ ]:
def modeltest(c, r, t = data): #c = classify model, r = regression model, t = test set
    rs = [] #r^2 values to return
    cps = c.predict(t.drop(columns = ["Flow", "Flow Class"])) #classify predictions
    rps = r.predict(t.drop(columns = ["Flow", "Flow Class"])) #regression predictions
    
    cps = pfix(cps) #fixes cps and rps into 1d arrays of floats, instead of 2d arrays of floats
    rps = pfix(rps)
     
    cps = np.where(cps  < .5, 0, 1) #forces predictions into classification of 1 or 0 (flow or no flow) (to be revised later)
    
    fps = cps * rps #final predictions (0 if no flow and regression prediction otherwise)
    
    exp = t["Flow"].to_numpy() #expected values
    cexp = t["Flow Class"].to_numpy() #classify expected values
    
    b = exp != 0
    rexp = exp[b] #regression expected values
    
    cstats = (cps * cexp) * 2
    cstats = cstats + (cexp - cps) #breaks results into one array of values 0, 1, -1, and 2 (specified below)
    cstats = np.where(cstats < 0, 3, cstats)
    
    csum = np.bincount(cstats.astype(int))
    tn = csum[0] #true negative, red
    fn = csum[1] #false negative, yellow
    fp = csum[3] #false positive, orange
    tp = csum[2] #true positive, green
    
    cp = tn + tp #correct predictions
    tot = tn + fn + fp + tp
    
    
    print("Classify results:")
    print("\nNegative Predictions: ", (tn + fn)/tot * 100, "%\tActual Negative: ", (tn + fp)/tot * 100, "%")
    print("Positive Predicitons: ", (tp + fp)/tot * 100, "%\tActual Positive: ", (tp + fn)/tot * 100, "%")
    print("\nTrue Negatives: ", tn, "\t", tn/tot * 100, "%")
    print("False Negatives: ", fn, "  ", fn/tot * 100, "%")
    print("False Positives: ", fp, "\t", fp/tot * 100, "%")
    print("True Positives: ", tp, "\t", tp/tot * 100, "%")
    print("\nNo Flow Predicitons: ", (tn/(tn + fn)) * 100, "% Correct")
    print("Flow Predictions: ", (tp/(tp + fp)) * 100, "% Correct")
    print("\nCorrect Predictions:", cp, "  ", cp/tot * 100, "%")
    R = np.corrcoef(cps, cexp)[1,0]
    R2 = R**2
    R = R * 100
    R2 = R2 * 100
    R = round(R, 4)
    R2 = round(R2, 4)
    rs.append(R2)
    print("R: ", R, "%  R^2: ", R2, "%")
    
    print("\n\nRegression Visualization and Statistics: ")
    
    colors = np.where(cstats == 0, "red", np.where(cstats == 1, "yellow", np.where(cstats == 3, "orange", "green")))
    rcb = colors != "orange" #regression color boolean
    rcolors = colors[rcb] #colors for flow powders only
    rcb = rcolors != "red"
    rcolors = rcolors[rcb]
    
    
    #scatter = plt.scatter(rps[b], rexp, c = rcolors, label = rcolors)
    
    cdict = {"yellow": "False Negative", "green": "True Positive", "red": "True Negative", "orange": "False Positive"}
    
    cax = np.empty(0)
    cay = np.empty(0)
    for g in ["yellow", "green"]:
        ix = np.where(colors == g)
        plt.scatter(rps[ix], exp[ix], c = colors[ix], label = cdict[g])
        cax = np.append(cax, rps[ix])
        cay = np.append(cay, exp[ix])
    R = np.corrcoef(cax, cay)[1,0]
    plt.legend()
    plt.xlabel("Predicted Flow")
    plt.ylabel("Expected Flow")
    plt.title("Flow Powders Regression Prediction")
    plt.show()
    R2 = R**2
    R = R * 100
    R2 = R2 * 100
    R = round(R, 4)
    R2 = round(R2, 4)
    rs.append(R2)
    print("R: ", R, "%  R^2: ", R2, "%")
    
    cax = np.empty(0)
    cay = np.empty(0)
    for g in ["green", "orange"]:
        ix = np.where(colors == g)
        plt.scatter(fps[ix], exp[ix], c = colors[ix], label = cdict[g]) #some of this can be made more efficient or modularized or something
        cax = np.append(cax, fps[ix])
        cay = np.append(cay, exp[ix])
    
    plt.xlabel("Predicted Flow")
    plt.ylabel("Expected Flow")
    plt.title("Predicted Flow Powder Regression Predictions vs Expected Flow")
    plt.legend()
    plt.show()
    R = np.corrcoef(cax, cay)[1,0]
    R2 = R**2
    R = R * 100
    R2 = R2 * 100
    R = round(R, 4)
    R2 = round(R2, 4)
    rs.append(R2)
    print("R: ", R, "%  R^2: ", R2, "%")
    
    cax = np.empty(0)
    cay = np.empty(0)
    for g in ["green", "orange", "yellow", "red"]:
        ix = np.where(colors == g)
        plt.scatter(fps[ix], exp[ix], c = colors[ix], label = cdict[g])
        cax = np.append(cax, fps[ix])
        cay = np.append(cay, exp[ix])
    R = np.corrcoef(cax, cay)[1,0]
    plt.xlabel("Predicted Flow")
    plt.ylabel("Expected Flow")
    plt.title("Final Prediction vs Final Expected")
    plt.legend()
    plt.show()
    R2 = R**2
    R = R * 100
    R2 = R2 * 100
    R = round(R, 4)
    R2 = round(R2, 4)
    rs.append(R2)
    print("R: ", R, "%  R^2: ", R2, "%")
    return rs

In [21]:
def pfix(arr): #it works (flips axes or something)
    fppp = np.array(arr)
    fppp = np.absolute(fppp)
    fpp = fppp > 0
    newfp = fppp[fpp]
    return newfp

In [ ]:
def trymodel(c, r, t = data, visualize = True, ctd = []): #returns 1x4 array containing 4 R^2 values. Classification, Regression, Predicted Regression, and Overall
    if len(ctd) != 0:    
        c = c.drop(columns = ctd)
        r = r.drop(columns = ctd)
        t = t.drop(columns = ctd)
    cmodel = makemodel(c.drop(columns = "Flow"), "Flow Class")
    rmodel = makemodel(r.drop(columns = "Flow Class"))
    if visualize:
        return modeltest(cmodel, rmodel, t)

In [ ]:
def coltest(cols, ctd = []): 
    plist = []
    for i in datalist:
        plist.append(i.sample(n = 2842))
    df = pd.concat(plist)
    df = df.sample(frac = 1)
    flowdf = df[df["Flow Class"] == 1]
    results = []
    datar = []
    dfr = []
    results.append(datar)
    results.append(dfr)
    for i in ctd:
        if i in cols:
            cols.remove(i)
    for i in cols:
        cttd = ctd.copy()
        cttd.append(i)
        print("Feature dropped: ", i)
        results[0].append([i, "Data", trymodel(df, flowdf, ctd = cttd)])
        results[1].append([i, "Df", trymodel(df, flowdf, df, ctd = cttd)])
        print("\n\n\n\n\n\n\n\n\n")
    return results